<a href="https://colab.research.google.com/github/nicolashernandez/teaching_nlp/blob/main/M2-ATAL-2021-22_02_Machine_Learning_by_feature_engineering_Use_case_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objectifs

* Vérifier les comparatifs de performance des systèmes états de l'art en reconnaissance d'entités nommées en les testant sur de nouvelles données  
* En utilisant le même corpus d'entraînement que les systèmes état de l'art, tenter de faire mieux avec une méthode d'apprentissage supervisée à base de traits.

Les systèmes état de l'art considérés sont spaCy et Stanza.

# Installation

Ne pas oublier de redémarrer l'environnement d'exécution après exécution du code ci-dessous.

In [ ]:
# Download the "small" french spacy nlp model
# Ne pas oublier de redémarrer l'environnement d'exécution
!python -m spacy download fr_core_news_sm

In [ ]:
# installation de stanza
!pip install stanza

In [ ]:
# retrieve a list of french stop words 
# "ça peut toujours servir..."
import nltk 
nltk.download('stopwords')
fr_stop_words = nltk.corpus.stopwords.words('french')

In [ ]:
# utilities 

def flatten(t):
  # applatie une liste de listes en une unique liste... 
  return [item for sublist in t for item in sublist]

import re 
def normalise_labels(sentences):
  # normalise les sorties des étiquettes NER utilisées par les différents 
  # systèmes afin de les rendre comparable
  new_sentences = list()
  for sentence in sentences:
    new_sentence = list()
    for label in sentence:
      if label != 'O':
        label = re.sub('^[A-Z]-','', label)
      new_sentence.append(label)
    new_sentences.append(new_sentence)
  return new_sentences

# Protocole d'évaluation

## WiNER, données de développement

[WiNER](https://github.com/YoannDupont/WiNER-fr) is a free corpus for Named Entity Recognition based on Wikinews texts.

The current named entity tagset is:
* Date (absolute dates.)
* Event (conferences, sports events, annual events, celebration days, named climatic events, etc.)
*  Hour (absolute hours. If present, they include time zones, UTC, GMT, etc.)
* Location (countries, towns, regions, addresses, astophysicals objects, hydrophysical objects, etc.)
* Organization (non profit organizations, companies, medias, etc.)
* Person (human individuals, without their title or function. They can be actual people or fictional characters.)
* Product (physical objects, brands, softwares.)

The tagset was defined using various campaigns or resources: MUC-6, CoNLL-2003, Named Entity annotated French Treebank and Quaero. Every type defined in this tagset is directly taken from one of those.

> Yoann Dupont. Un corpus libre, évolutif et versionné en entités nommées du français. TALN 2019 - Traitement Automatique des Langues Naturelles, Jul 2019, Toulouse, France. 
https://hal.archives-ouvertes.fr/hal-02448590

Le système d'annotation originale est de type **_stand off annotation_**. _standoff_ désigne les formats où les annotations sont stockées séparément du contenu annoté, lequel n’est jamais modifié par l'outil annotateur. En général les annotations référence les contenus annotés via des offsets (indice de caractères de début et de fin) ou des indices de tokens. La lecture de ces formats par un humain n’est pas forcément aisée, leur manipulation (édition et visualisation) requièrent des développements spécifiques mais ils présentent l’intérêt de gérer la concurrence entre annotations, de ne pas être contraint par la nature de l’information ajoutée (span, relation, ...) et surtout de ne perdre aucune information du document original (ne serait ce que sa mis en forme). 

Ici un [exemple de texte extrait du corpus](https://github.com/YoannDupont/WiNER-fr/blob/master/2016/01/2016_01_01-001.txt) et son [fichier associé qui contient les annotations "standoff"](https://github.com/YoannDupont/WiNER-fr/blob/master/2016/01/2016_01_01-001.ann).


### QUESTION

* Le corpus Winer original utilise-t-il des offsets ou des indices de tokens ? 

### VOTRE REPONSE

**TODO**

### Récupération et préparation des données

Pour les besoins de ce travail, nous utiliserons un extrait du corpus, tokenisé par spaCy, avec les annotations projetées sur les tokens. 

Ce sont ces classes LOC, ORG, PER et MISC qui nous intéresseront.

La classe _Location_ a été renommée en _LOC_, _Organization_ en _ORG_, _Person_ en _PER_. Stanza et spaCy reconnaissent aussi des entités de type MISC. Sans vraiment chercher ce qu'elle désigne, nous avons pris le partie de renommer _Product_ en _MISC_. 



In [ ]:
!mkdir -p data
!wget -nc https://github.com/nicolashernandez/teaching_nlp/raw/main/data/winer_dev.joblib -P data

In [ ]:
# load the test corpus
from joblib import load
winer_corpus = load('data/winer_dev.joblib')

# get the tokens of each text
winer_ref = [[label for token, pos, label in text] for text in winer_corpus]
labels = list(set(flatten(winer_ref)))
winer_tokens = [[token for token, pos, label in text] for text in winer_corpus]

#
print ('#texts:', len(winer_corpus))
print ('labels:', labels)

print ('sample of annotated texts:', winer_corpus[0])   
print ('sample of tokenized text:', winer_tokens[0])   

## Métriques d'évaluation

La reconnaissance des entités nommées est un problème de classification multi-classes. On peut donc utiliser toutes les mesures de classification. En général, on privilégie la **micro-moyenne de scores F1** (_micro-averaged F1_). 

La micro-moyenne d'une métrique aggrège d'abord les contributions de toutes les classes avant de calculer la métrique tandis que la macro-moyenne calcule la métrique indépendamment pour chaque classe puis en fait la moyenne. 

La macro-moyenne traite toutes les classes de manière égale. 
Dans le cas où le système n'est bon que pour reconnaître une classe (la classe dominante), la micro-moyenne peut être élevée car les contributions des autres classes seront insignifiantes. Dans ce cas, la macro-moyenne sera basse.

Dans le cas où le système n'est mauvais que pour la reconnaissance d'une seule classe, la macro-moyenne gommera son effet. La micro-moyenne peut être élevée même si le système donne de mauvais résultat sur des classes rares, car il donne plus de poids aux classes communes.




In [ ]:
# Measures definition
from sklearn.metrics import classification_report

def results_per_class(labels, y_ref, y_hyp):
  # Inspect per-class results in more detail:
  sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
  )
  # print ('y_ref', len(y_ref), 'y_hyp', len(y_hyp), 'sorted_labels', len(sorted_labels))
  return classification_report(flatten(y_ref), flatten(y_hyp), labels=sorted_labels, digits=3)

In [ ]:
# Il y a beaucoup plus d'entités 'O' que les autres dans le corpus, 
# mais nous sommes davantage intéressés par les autres entités. 
# Pour ne pas biaiser les scores de moyenne, on retire les étiquettes qui ne nous intéressent pas.
print ("before removing:", labels)
labels_to_remove = ['O', 'Event', 'Date', 'Hour']
for l in labels_to_remove:
  if l in labels:
    labels.remove(l)
print ("after removing:", labels)

# Prédiction et évaluation de spaCy


## Construction de la chaîne de traitements

In [ ]:
# Create my whitespace tokenizer
#
# one trick to make spaCy consider a tokenization performed by a third-party component: 
# use whitespace as separator for marking the third-party component tokenization 
# and force spacy to use a whitespace tokenizer...

from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(' ')
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

In [ ]:
# Build a spacy nlp pipeline with a ner model and the whitespace tokenizer

import spacy

# load a nlp model
spacy_nlp = spacy.load("fr_core_news_sm")

# declare the whitespace tokenizer
spacy_nlp.tokenizer = WhitespaceTokenizer(spacy_nlp.vocab)

#
#print ('nlp pipeline='+str([name for (name, proc) in spacy_nlp.pipeline]))
# pipeline['tagger', 'parser', 'ner']

## Prédiction

In [ ]:
# Run spacy nlp pipeline made of ner component and evaluate it

#
import time
start_time = time.time()

# 
spacy_hyp = []
for text in winer_tokens:
    doc = spacy_nlp(' '.join(text), disable=["tagger", "parser"])
    #print (doc)
    spacy_hyp.append([(token.ent_iob_+'-'+token.ent_type_) for token in doc])
    #break

#
print("--- %s seconds ---" % (time.time() - start_time))

# normalize the hyp labels
normalized_spacy_hyp = normalise_labels(spacy_hyp)
#print (winer_tokens[0])
#print (winer_ref[0])
#print (spacy_hyp[0])
#print (normalized_spacy_hyp[0])


## Evaluation

In [ ]:
# Evaluate on data 
print (results_per_class(labels, winer_ref, normalized_spacy_hyp))

### QUESTION

La colonne _support_ indique le nombre d'instances par classe. 

* Le f1-score de la classe _MISC_ est très mauvais. Pour rappel, la classe MISC à reconnaître dans le corpus WiNER correspond à sa classe Product que nous avons renommé. Pour rappel encore, spaCy a été entraîné avec le corpus Wikiner. En vous rapportant à la définition de la classe Product du [corpusWiNER](https://github.com/YoannDupont/WiNER-fr) et en la comparant avec la définition de la classe MISC du [corpus Wikiner](https://www.sciencedirect.com/science/article/pii/S0004370212000276?via%3Dihub
), émettez une hypothèse qui puisse expliquer la mauvaise qualité des résultats.  
* Retirez la classe MISC du décompte ou bien vérifiez votre hypothèse.

### VOTRE REPONSE

**TODO**

# Prédiction et évaluation de Stanza


## Construction de la chaîne de traitements

In [ ]:
# Build a stanza nlp pipeline with a ner model and the whitespace tokenizer
# --- 1848.740861415863 seconds ---

import stanza

# download French model
stanza.download('fr') 

# https://stanfordnlp.github.io/stanza/tokenize.html#start-with-pretokenized-text 
# pretokenized (and sentence split) text correspond to use newline (\n) to separated sentences and each sentence is space separated tokens
stanza_nlp = stanza.Pipeline('fr', processors='tokenize,ner', tokenize_pretokenized=True) # initialize French neural pipeline

# run on sample 
#doc = stanza_nlp("Barack Obama est né à Hawaii .") # run annotation over a sentence
#print(doc)
#print(doc.entities)
#print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens], sep='\n')

## Prédiction

In [ ]:
# Running stanza nlp pipeline made of ner component and evaluate it

#
import time
start_time = time.time()

# 
stanza_hyp = []
for text in winer_tokens:
    doc = stanza_nlp(' '.join(text))
    stanza_hyp.append([token.ner for sent in doc.sentences for token in sent.tokens])

#
print("--- %s seconds ---" % (time.time() - start_time))

# normalize the hyp labels
normalized_stanza_hyp = normalise_labels(stanza_hyp)


Comme stanza prend un certain temps d'exécution... vous pouvez directement charger la prédiction précédemment sauvée pour vous...

In [ ]:
# dump
#from joblib import dump
#dump(normalized_stanza_hyp, 'data/winer_stanza_hyp.joblib')

#load
!mkdir -p data
!wget -nc https://github.com/nicolashernandez/teaching_nlp/raw/main/data/winer_stanza_hyp.joblib -P data
from joblib import load
normalized_stanza_hyp = load('data/winer_stanza_hyp.joblib')


## Evaluation

In [ ]:
# Evaluate on data 
print (results_per_class(labels, winer_ref, normalized_stanza_hyp))

### QUESTION

* En termes de rapidité de traitement, quel est le meilleur des deux systèmes spaCy ou Stanza ? 
* En termes de qualité de traitement,  quel est le meilleur des deux systèmes spaCy ou Stanza ?
* Est-ce que ces observations correspondent à ce qui est relevé sur le [comparatif de performance du site de spaCy](https://spacy.io/usage/facts-figures) ? 

### VOTRE REPONSE

**TODO**

# Construction d'un système NER selon une approche supervisée (CRF) à base de traits 

Les cellules ci-dessous implémentent une solution de bout en bout pour
- entraîner un modèle de prédiction à partir des données d'entraînement wikiner; 
- appliquer le modèle construit pour la prédiction d'entités nommées sur le corpus de dev winer;
- évaluer les résultats.

Avant de la mettre en oeuvre, lisez la suite.

Dans une méthode d'apprentissage supervisée à base de traits, c'est au "scientifique" ou "ingénieur de la donnée" de définir et d'implémenter les traits à observer dans la donnée et à donner en entrée du système d'apprentissage.

Les premières cellules de code ci-dessous proposent une implémentation d'une méthode d'extraction de traits du mot courant à savoir la méthode `word2features`.
Cette méthode doit être appelées aussi bien sur les données d'entraînement que sur les données où l'on cherche à appliquer un modèle construit pour obtenir une prédiction.

Accessoirement l'algorithme d'apprentissage est un *CRF (Conditional Random Fields)* de l'implémentation `sklearn_crfsuite.CRF`.
L'implémentation du système ci-dessous suit la démarche décrite dans le turtoriel de l'implémentation [sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html).
L'optimisation des paramètres est possible (cf. le tutoriel).

Poursuivez votre lecture mais sachez que le système fonctionne en l'état.

## Wikiner, données d'entraînement

Pour entraîner notre système, nous allons utiliser les mêmes données d'entraînement de que les composants NER de spaCy ou Stanza à savoir les données du corpus Wikiner.

```
@Article{nothman2012:artint:wikiner,
  author = {Joel Nothman and Nicky Ringland and Will Radford and Tara Murphy and James R. Curran},
  title = {Learning multilingual named entity recognition from {Wikipedia}},
  journal = {Artificial Intelligence},
  publisher = {Elsevier},
  volume = {194},
  pages = {151--175},
  year = {2012},
  doi = {10.1016/j.artint.2012.03.006},
  url = {http://dx.doi.org/10.1016/j.artint.2012.03.006}
}
```
https://www.sciencedirect.com/science/article/pii/S0004370212000276?via%3Dihub

Les données utilisées proviennent de l'annotation "silver standard" du [corpus Wikiner généré à partir de la Wikipedia et téléchargeable ici](https://github.com/dice-group/FOX/tree/master/input/Wikiner). 

Le pos tagging a été modifié pour adopter le système d'étiquettes de l'[universal dependencies (UD)](https://universaldependencies.org/u/pos/) (grâce à spaCy).


In [ ]:
# Getting the data
!mkdir -p data 
!wget -nc https://github.com/nicolashernandez/teaching_nlp/raw/main/data/wikiner_ud.joblib.bz2 -P data
!bzip2 -dk data/wikiner_ud.joblib.bz2

# Loading the corpus 
from joblib import load
wikiner_corpus = load('data/wikiner_ud.joblib') 


# Aperçu du nombre de phrases et d'une phrase annotée (liste de tokens composés de la forme, de la catégorie grammaticale et de l'étiquette BIO correspondant en l'entité nommée.
print (len(wikiner_corpus))
print (wikiner_corpus[0])  
# [('Il', 'PRO:PER', 'O'), ('assure', 'VER:pres', 'O'), ('à', 'VER:pper', 'O'), ('la', 'DET:ART', 'O'), ('suite', 'NOM', 'O'), ('de', 'PRP', 'I-PER'), ('Saussure', 'NAM', 'I-PER'), ('le', 'DET:ART', 'O'), ('cours', 'NOM', 'O'), ('de', 'PRP', 'O'), ('grammaire', 'NOM', 'O'), ('comparée', 'ADJ', 'O'), (',', 'PUN', 'O'), ("qu'", 'PRO:REL', 'O'), ('il', 'PRO:PER', 'O'), ('complète', 'VER:subp', 'O'), ('à', 'VER:pper', 'O'), ('partir', 'VER:infi', 'O'), ('de', 'PRP', 'O'), ('1894', 'NUM', 'O'), ('par', 'PRP', 'O'), ('une', 'DET:ART', 'O'), ('conférence', 'NOM', 'O'), ('sur', 'PRP', 'O'), ("l'", 'DET:ART', 'O'), ('iranien', 'ADJ', 'O'), ('.', 'SENT', 'O')]
# [('Il', 'PRON', 'O'), ('assure', 'VERB', 'O'), ('à', 'ADP', 'O'), ('la', 'DET', 'O'), ('suite', 'NOUN', 'O'), ('de', 'ADP', 'I-PER'), ('Saussure', 'NOUN', 'I-PER'), ('le', 'DET', 'O'), ('cours', 'NOUN', 'O'), ('de', 'ADP', 'O'), ('grammaire', 'ADJ', 'O'), ('comparée', 'VERB', 'O'), (',', 'PUNCT', 'O'), ("qu'", 'SCONJ', 'O'), ('il', 'PRON', 'O'), ('complète', 'VERB', 'O'), ('à', 'ADP', 'O'), ('partir', 'VERB', 'O'), ('de', 'ADP', 'O'), ('1894', 'NUM', 'O'), ('par', 'ADP', 'O'), ('une', 'DET', 'O'), ('conférence', 'NOUN', 'O'), ('sur', 'ADP', 'O'), ("l'", 'DET', 'O'), ('iranien', 'NOUN', 'O'), ('.', 'PUNCT', 'O')]

Dans la cellule ci-dessous vous pouvez spécifier la quantité de données du corpus que vous souhaitez utiliser pour votre entraînement. En l'état, il est spécifié 10% pour faire des tests fonctionnels.

In [ ]:
# shuffle the data
#from random import shuffle
#shuffle(wikiner_corpus)

print ('len(wikiner_corpus):', len(wikiner_corpus))


# compute corpus partition sizes 
_100_percent = int(len(wikiner_corpus)/100*100)

_80_percent = int(len(wikiner_corpus)/100*80)
_50_percent = int(len(wikiner_corpus)/100*50)
_10_percent = int(len(wikiner_corpus)/100*10)

# ICI vous pouvez choisir la quantité de données 
# 10 % vous permet de faire des tests fonctionnels
train_sentences = wikiner_corpus[:_10_percent]
print ('len(train_sentences):', len(train_sentences))


## Extraction des caractéristiques

Ci-dessous vous trouverez la méthode `word2features` qui se charge de transformer les mots en traits que prendra l'algorithme d'apprentissage en entrée.



### QUESTION

* Exécutez les différentes cellules jusqu'à l'évaluation. Optez aussi pour un entraînement avec 100 % des données d'entrainement. Les performances produites par les traits disponibles constitueront votre "baseline". En attendant que les traitements se terminent (quelques minutes à 100%), répondez aux questions suivantes.
* Dans la méthode `word2features`, que signifient `word`, `postag`, `wordminus1`, `postagminus1`, `wordplus1`, `postagplus1` ?
* Dans la méthode `word2features`, que signifient les traits `word.lower()`,    `word[-3:]`, `word.isupper()`, `word.istitle()`, `word.isdigit()`, `postag`, `postag[:2]` ?
* Reprenez la méthode, ajoutez de nouveaux traits, réécrivez les définitions existantes, votre objectif obtenir le meilleur micro-average f1 score sur winer avant le temps imparti et délivrer votre modèle dans le dépôt indiqué par l'enseignant. Expliquer verbeusement les traits que vous implémentez ! La mise au point de votre jeu de traits doit être éprouver dans l'env gcolab avec les ressources limitées (RAM à 12 Go).
* Comparez vos performances avec votre baseline et avec les systèmes état de l'art, que pouvez-vous conclure ? 

### VOTRE REPONSE

**TODO**

In [ ]:
# Features definition
#
# https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html
#
# Feature extraction
# POS tags can be seen as pre-extracted features. 
# Let’s extract more features (word parts, simplified POS tags, lower/title/upper flags, features of nearby words) 
# and convert them to sklear-crfsuite format - each sentence should be converted to a list of dicts. 
# This is a very simple baseline; you certainly can do better.
# https://docs.python.org/3/library/stdtypes.html

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        wordminus1 = sent[i-1][0]
        postagminus1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': wordminus1.lower(),
            '-1:word.istitle()': wordminus1.istitle(),
            '-1:word.isupper()': wordminus1.isupper(),
            '-1:postag': postagminus1,
            '-1:postag[:2]': postagminus1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        wordplus1 = sent[i+1][0]
        postagplus1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': wordplus1.lower(),
            '+1:word.istitle()': wordplus1.istitle(),
            '+1:word.isupper()': wordplus1.isupper(),
            '+1:postag': postagplus1,
            '+1:postag[:2]': postagplus1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]


In [ ]:
# Features extraction

tokens_train = [sent2tokens(s) for s in train_sentences]
X_train = [sent2features(s) for s in train_sentences]
y_train = normalise_labels([sent2labels(s) for s in train_sentences])

print (tokens_train[0])
print (X_train[0][0])
print (X_train[0][1])
print (len(y_train))
print (y_train[0])

## Entraînement

In [ ]:
# instanciation du modèle crf et initialisation des hyperparamètres
!pip install sklearn_crfsuite

In [ ]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)

In [ ]:
import time
#
start_time = time.time()

# train
crf.fit(X_train, y_train);

#
print("--- %s seconds ---" % (time.time() - start_time)) #  110s - 230s


In [ ]:
# Persistence: model dump
!mkdir -p models

from joblib import dump
dump(crf, 'models/wikiner_sklearn-crfsuite.joblib')

## Prédiction

In [ ]:
# Extract features from the test data and predict 
winer_dev = [sent2features(s) for s in winer_corpus]

crf_hyp = crf.predict(winer_dev)

## Evaluation

In [ ]:
# Evaluate on data 
print (results_per_class(labels, winer_ref, crf_hyp))

## Vérification de ce que le classifieur a appris
https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-check-what-classifier-learned

Ce qui suit peut vous donner des idées sur le genre de traits qui peut vous être utiles

In [ ]:
# quid des transitions d'une étiquette à l'autre ?
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
# quid des caractéristiques ? 
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive features:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative features:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])